### Helper Scripts for programming automated testing for a 16-bit ALU on the MOJO FPGA

Below are the scripts used to generate the repeated portions of the alu_tester_auto module:

In [1]:
def instantiate_states(no_states):
    tests = ""
    
    for i in range(no_states):
        tests += f", TEST_{i}"
    
    return "fsm autostate = {STANDBY" + tests + "};"

In [2]:
instantiate_states(3)

'fsm autostate = {STANDBY, TEST_0, TEST_1, TEST_2};'

In [3]:
def instantiate_tester (test_no, input_a, input_b, opcode, expected):
    return f"alu_statement_tester test_{test_no} (#INPUT_A(16h{input_a}), #INPUT_B(16h{input_b}), #OPCODE(6b{opcode}), #EXPECTED(16h{expected}));"

In [4]:
instantiate_tester(0, "abcd", "ef42", "000000", "9b36")

'alu_statement_tester test_0 (#INPUT_A(16habcd), #INPUT_B(16hef42), #OPCODE(6b000000), #EXPECTED(16h9b36));'

In [11]:
def state_case(state_no, is_final):
    output = ""
    output += f"\tautostate.TEST_{state_no}:\n"
    output += f"\t\tio_seg = test_{state_no}.io_seg;\n"
    output += f"\t\tio_sel = test_{state_no}.io_sel;\n"
    output += f"\t\topcode_led = test_{state_no}.opcode_led;\n"
    output += f"\t\ttest_{state_no}.start = 1;\n"
    output += f"\t\tif(test_{state_no}.done)\n"
    if (is_final):
        output += f"\t\t\tautostate.d = autostate.STANDBY;"
    else:
        output += f"\t\t\tautostate.d = autostate.TEST_{state_no + 1};"
    
    return output

In [6]:
print(state_case(1,False))

	autostate.TEST_1:
		io_seg = test_1.io_seg;
		io_sel = test_1.io_sel;
		opcode_led = test_1.opcode_led;
		test_1.start = 1;
		if(test_1.done)
			autostate.d = autostate.TEST_2


### Test Cases
Test cases are in the format (a_input, b_input, opcode, expected_output):

In [17]:
test_cases = [
    ("abcd", "ef42", "000000", "9b0f"), #ADD
    ("0168", "0168", "000000", "02d0"), #ADD
    ("0168", "0000", "000001", "0168"), #SUB
    ("abcd", "abcd", "000001", "0000"), #SUB
    ("0000", "ffff", "011000", "0000"), #AND
    ("ffff", "ffff", "011000", "ffff"), #AND
    ("0000", "ffff", "011110", "ffff"), #OR
    ("0001", "0010", "011110", "0011"), #OR
    ("ffff", "ffff", "010110", "0000"), #XOR
    ("ffff", "0000", "010110", "ffff"), #XOR
    ("2121", "4242", "011010", "2121"), #A
    ("4242", "4242", "011010", "4242"), #A
    ("ffff", "0001", "100000", "FFFE"), #SHL
    ("ffff", "0008", "100000", "FF00"), #SHL
    ("ffff", "0001", "100001", "7FFF"), #SHR
    ("ffff", "0008", "100001", "00FF"), #SHR
    ("ffff", "0001", "100011", "FFFF"), #SRA
    ("3fff", "0008", "100011", "003F"), #SRA
    ("7fff", "7fff", "000010", "0001"), #MUL
    ("7fff", "0000", "000010", "0000"), #MUL 
    ("7fff", "7fff", "110011", "0001"), #CMPEQ
    ("7fff", "0000", "110011", "0000"), #CMPEQ
    ("7fff", "7fff", "110101", "0000"), #CMPLT 
    ("0000", "7fff", "110101", "0001"), #CMPLT
    ("7fff", "0000", "110111", "0000"), #CMPLE
    ("2121", "2121", "110111", "0001"), #CMPLE
    ("2af0", "afe0", "010001", "d51f"), #NAND
    ("ff00", "00ff", "010001", "ffff"), #NAND
    ("2af0", "afe0", "011111", "500f"), #NOR
    ("ff00", "00ff", "011111", "0000"), #NOR
    ("2af0", "afe0", "010111", "7aef"), #NXOR
    ("ff00", "00ff", "010111", "0000"), #NXOR  
    ("001f", "0015", "000111", "000A"), #MODULO
    ("002A", "0000", "000111", "002A"), #MODULO 
]

Generate a list of states for alu_test_auto & instantiate the statement tester modules: 

In [18]:
l = len(test_cases)

print(instantiate_states(l))

for i in range(l):
    print(instantiate_tester(i, test_cases[i][0], test_cases[i][1], test_cases[i][2], test_cases[i][3]))

fsm autostate = {STANDBY, TEST_0, TEST_1, TEST_2, TEST_3, TEST_4, TEST_5, TEST_6, TEST_7, TEST_8, TEST_9, TEST_10, TEST_11, TEST_12, TEST_13, TEST_14, TEST_15, TEST_16, TEST_17, TEST_18, TEST_19, TEST_20, TEST_21, TEST_22, TEST_23, TEST_24, TEST_25, TEST_26, TEST_27, TEST_28, TEST_29, TEST_30, TEST_31, TEST_32, TEST_33};
alu_statement_tester test_0 (#INPUT_A(16habcd), #INPUT_B(16hef42), #OPCODE(6b000000), #EXPECTED(16h9b0f));
alu_statement_tester test_1 (#INPUT_A(16h0168), #INPUT_B(16h0168), #OPCODE(6b000000), #EXPECTED(16h02d0));
alu_statement_tester test_2 (#INPUT_A(16h0168), #INPUT_B(16h0000), #OPCODE(6b000001), #EXPECTED(16h0168));
alu_statement_tester test_3 (#INPUT_A(16habcd), #INPUT_B(16habcd), #OPCODE(6b000001), #EXPECTED(16h0000));
alu_statement_tester test_4 (#INPUT_A(16h0000), #INPUT_B(16hffff), #OPCODE(6b011000), #EXPECTED(16h0000));
alu_statement_tester test_5 (#INPUT_A(16hffff), #INPUT_B(16hffff), #OPCODE(6b011000), #EXPECTED(16hffff));
alu_statement_tester test_6 (#INPUT

In [19]:
l = len(test_cases)
for i in range(l):
    print(f"test_{i}.start = 0;")

test_0.start = 0;
test_1.start = 0;
test_2.start = 0;
test_3.start = 0;
test_4.start = 0;
test_5.start = 0;
test_6.start = 0;
test_7.start = 0;
test_8.start = 0;
test_9.start = 0;
test_10.start = 0;
test_11.start = 0;
test_12.start = 0;
test_13.start = 0;
test_14.start = 0;
test_15.start = 0;
test_16.start = 0;
test_17.start = 0;
test_18.start = 0;
test_19.start = 0;
test_20.start = 0;
test_21.start = 0;
test_22.start = 0;
test_23.start = 0;
test_24.start = 0;
test_25.start = 0;
test_26.start = 0;
test_27.start = 0;
test_28.start = 0;
test_29.start = 0;
test_30.start = 0;
test_31.start = 0;
test_32.start = 0;
test_33.start = 0;


Generate the states and state transitions for the alu_tester_auto:

In [20]:
l = len(test_cases)
for i in range(l-1):
    print(state_case(i,False))
    print()

print(state_case(l-1, True))

	autostate.TEST_0:
		io_seg = test_0.io_seg;
		io_sel = test_0.io_sel;
		opcode_led = test_0.opcode_led;
		test_0.start = 1;
		if(test_0.done)
			autostate.d = autostate.TEST_1;

	autostate.TEST_1:
		io_seg = test_1.io_seg;
		io_sel = test_1.io_sel;
		opcode_led = test_1.opcode_led;
		test_1.start = 1;
		if(test_1.done)
			autostate.d = autostate.TEST_2;

	autostate.TEST_2:
		io_seg = test_2.io_seg;
		io_sel = test_2.io_sel;
		opcode_led = test_2.opcode_led;
		test_2.start = 1;
		if(test_2.done)
			autostate.d = autostate.TEST_3;

	autostate.TEST_3:
		io_seg = test_3.io_seg;
		io_sel = test_3.io_sel;
		opcode_led = test_3.opcode_led;
		test_3.start = 1;
		if(test_3.done)
			autostate.d = autostate.TEST_4;

	autostate.TEST_4:
		io_seg = test_4.io_seg;
		io_sel = test_4.io_sel;
		opcode_led = test_4.opcode_led;
		test_4.start = 1;
		if(test_4.done)
			autostate.d = autostate.TEST_5;

	autostate.TEST_5:
		io_seg = test_5.io_seg;
		io_sel = test_5.io_sel;
		opcode_led = test_5.opcode_led;
